# 9

In [ ]:
import random
from PIL import Image, ImageDraw, ImageFont

class Space:
    def __init__(self, height, width, num_hospitals):
        """Create a new state space with given dimensions."""
        self.height = height
        self.width = width
        self.num_hospitals = num_hospitals
        self.houses = set()
        self.hospitals = set()

    def add_house(self, row, col):
        """Add a house at a particular location in state space."""
        self.houses.add((row, col))

    def available_spaces(self):
        """Returns all cells not currently used by a house or hospital."""
        # Consider all possible cells
        candidates = set(
            (row, col)
            for row in range(self.height)
            for col in range(self.width)
        )

        # Remove all houses and hospitals
        for house in self.houses:
            candidates.remove(house)
        for hospital in self.hospitals:
            candidates.remove(hospital)
        return candidates

    def hill_climb(self, maximum=None, image_prefix=None, log=False):
        """Performs hill-climbing to find a solution."""
        count = 0

        # Start by initializing hospitals randomly
        self.hospitals = set()
        for i in range(self.num_hospitals):
            self.hospitals.add(random.choice(list(self.available_spaces())))
        if log:
            print("Initial state: cost", self.get_cost(self.hospitals))
        if image_prefix:
            self.output_image(f"{image_prefix}{str(count).zfill(3)}.png")

        # Continue until we reach maximum number of iterations
        while maximum is None or count < maximum:
            count += 1
            best_neighbors = []
            best_neighbor_cost = None

            # Consider all hospitals to move
            for hospital in self.hospitals:
                # Consider all neighbors for that hospital
                for replacement in self.get_neighbors(*hospital):
                    # Generate a neighboring set of hospitals
                    neighbor = self.hospitals.copy()
                    neighbor.remove(hospital)
                    neighbor.add(replacement)

                    # Check if neighbor is best so far
                    cost = self.get_cost(neighbor)
                    if best_neighbor_cost is None or cost < best_neighbor_cost:
                        best_neighbor_cost = cost
                        best_neighbors = [neighbor]
                    elif best_neighbor_cost == cost:
                        best_neighbors.append(neighbor)

            # None of the neighbors are better than the current state
            if best_neighbor_cost >= self.get_cost(self.hospitals):
                return self.hospitals

            # Move to a highest-valued neighbor
            else:
                if log:
                    print(f"Found better neighbor: cost {best_neighbor_cost}")
                self.hospitals = random.choice(best_neighbors)

            # Generate image
            if image_prefix:
                self.output_image(f"{image_prefix}{str(count).zfill(3)}.png")

    def get_cost(self, hospitals):
        """Calculates sum of distances from houses to nearest hospital."""
        cost = 0
        for house in self.houses:
            cost += min(
                abs(house[0] - hospital[0]) + abs(house[1] - hospital[1])
                for hospital in hospitals
            )
        return cost

    def get_neighbors(self, row, col):
        """Returns neighbors not already containing a house or hospital."""
        candidates = [
            (row - 1, col),
            (row + 1, col),
            (row, col - 1),
            (row, col + 1)
        ]
        neighbors = []
        for r, c in candidates:
            if (r, c) in self.houses or (r, c) in self.hospitals:
                continue
            if 0 <= r < self.height and 0 <= c < self.width:
                neighbors.append((r, c))
        return neighbors

    def output_image(self, filename):
        """Generates image with all houses and hospitals."""
        cell_size = 100
        cell_border = 2
        cost_size = 40
        padding = 10

        # Create a blank canvas
        img = Image.new(
            "RGBA",
            (self.width * cell_size,
             self.height * cell_size + cost_size + padding * 2),
            (255, 255, 255, 255)  # White background with full opacity
        )
        house = Image.open("/content/H.jpg").convert("RGBA").resize(
            (cell_size, cell_size)
        )
        hospital = Image.open("/content/H+.jpg").convert("RGBA").resize(
            (cell_size, cell_size)
        )
        font = ImageFont.truetype("/content/OpenSans-Regular.ttf", 30)
        draw = ImageDraw.Draw(img)

        for i in range(self.height):
            for j in range(self.width):
                # Draw cell
                rect = [
                    (j * cell_size + cell_border,
                     i * cell_size + cell_border),
                    ((j + 1) * cell_size - cell_border,
                     (i + 1) * cell_size - cell_border)
                ]
                draw.rectangle(rect, fill="black")

                if (i, j) in self.houses:
                    img.paste(house, rect[0], house)  # Use house image as mask
                if (i, j) in self.hospitals:
                    img.paste(hospital, rect[0], hospital)  # Use hospital image as mask

        # Add cost
        draw.rectangle(
            (0, self.height * cell_size, self.width * cell_size,
             self.height * cell_size + cost_size + padding * 2),
            "black"
        )
        draw.text(
            (padding, self.height * cell_size + padding),
            f"Cost: {self.get_cost(self.hospitals)}",
            fill="white",
            font=font
        )

        img.save(filename)


# Create a new space and add houses randomly
s = Space(height=10, width=20, num_hospitals=3)
for i in range(6):
    s.add_house(random.randrange(s.height), random.randrange(s.width))

# Use local search to determine hospital placement
hospitals = s.hill_climb(image_prefix="hospitals", log=True)


Initial state: cost 34
Found better neighbor: cost 31
Found better neighbor: cost 30
Found better neighbor: cost 29
Found better neighbor: cost 28
Found better neighbor: cost 27
Found better neighbor: cost 26


In [ ]:
import random
from PIL import Image, ImageDraw, ImageFont

class Space:
    def __init__(self, height, width, num_hospitals):
        """Create a new state space with given dimensions."""
        self.height = height
        self.width = width
        self.num_hospitals = num_hospitals
        self.houses = set()
        self.hospitals = set()

    def add_house(self, row, col):
        """Add a house at a particular location in state space."""
        self.houses.add((row, col))

    def available_spaces(self):
        """Returns all cells not currently used by a house or hospital."""
        # Consider all possible cells
        candidates = set(
            (row, col)
            for row in range(self.height)
            for col in range(self.width)
        )

        # Remove all houses and hospitals
        for house in self.houses:
            candidates.remove(house)
        for hospital in self.hospitals:
            candidates.remove(hospital)
        return candidates

    def random_start(self):
        """Initialize hospitals randomly."""
        self.hospitals = set(random.sample(self.available_spaces(), self.num_hospitals))

    def hill_climb(self, maximum=None, image_prefix=None, log=False):
        """Performs hill-climbing to find a solution."""
        count = 0

        # Initialize hospitals randomly
        self.random_start()
        if log:
            print("Initial state: cost", self.get_cost(self.hospitals))
        if image_prefix:
            self.output_image(f"{image_prefix}{str(count).zfill(3)}.png")

        # Continue until we reach maximum number of iterations
        while maximum is None or count < maximum:
            count += 1
            best_neighbors = []
            best_neighbor_cost = None

            # Consider all hospitals to move
            for hospital in self.hospitals:
                # Consider all neighbors for that hospital
                for replacement in self.get_neighbors(*hospital):
                    # Generate a neighboring set of hospitals
                    neighbor = self.hospitals.copy()
                    neighbor.remove(hospital)
                    neighbor.add(replacement)

                    # Check if neighbor is best so far
                    cost = self.get_cost(neighbor)
                    if best_neighbor_cost is None or cost < best_neighbor_cost:
                        best_neighbor_cost = cost
                        best_neighbors = [neighbor]
                    elif best_neighbor_cost == cost:
                        best_neighbors.append(neighbor)

            # None of the neighbors are better than the current state
            if best_neighbor_cost >= self.get_cost(self.hospitals):
                return self.hospitals

            # Move to a highest-valued neighbor
            else:
                if log:
                    print(f"Found better neighbor: cost {best_neighbor_cost}")
                self.hospitals = random.choice(best_neighbors)

            # Generate image
            if image_prefix:
                self.output_image(f"{image_prefix}{str(count).zfill(3)}.png")

    def get_cost(self, hospitals):
        """Calculates sum of distances from houses to nearest hospital."""
        cost = 0
        for house in self.houses:
            cost += min(
                abs(house[0] - hospital[0]) + abs(house[1] - hospital[1])
                for hospital in hospitals
            )
        return cost

    def get_neighbors(self, row, col):
        """Returns neighbors not already containing a house or hospital."""
        candidates = [
            (row - 1, col),
            (row + 1, col),
            (row, col - 1),
            (row, col + 1)
        ]
        neighbors = []
        for r, c in candidates:
            if (r, c) in self.houses or (r, c) in self.hospitals:
                continue
            if 0 <= r < self.height and 0 <= c < self.width:
                neighbors.append((r, c))
        return neighbors

    def output_image(self, filename):
        """Generates image with all houses and hospitals."""
        cell_size = 100
        cell_border = 2
        cost_size = 40
        padding = 10

        # Create a blank canvas
        img = Image.new(
            "RGBA",
            (self.width * cell_size,
             self.height * cell_size + cost_size + padding * 2),
            (255, 255, 255, 255)  # White background with full opacity
        )
        house = Image.open("/content/H.jpg").convert("RGBA").resize(
            (cell_size, cell_size)
        )
        hospital = Image.open("/content/H+.jpg").convert("RGBA").resize(
            (cell_size, cell_size)
        )
        font = ImageFont.truetype("/content/OpenSans-Regular.ttf", 30)
        draw = ImageDraw.Draw(img)

        for i in range(self.height):
            for j in range(self.width):
                # Draw cell
                rect = [
                    (j * cell_size + cell_border,
                     i * cell_size + cell_border),
                    ((j + 1) * cell_size - cell_border,
                     (i + 1) * cell_size - cell_border)
                ]
                draw.rectangle(rect, fill="black")

                if (i, j) in self.houses:
                    img.paste(house, rect[0], house)  # Use house image as mask
                if (i, j) in self.hospitals:
                    img.paste(hospital, rect[0], hospital)  # Use hospital image as mask

        # Add cost
        draw.rectangle(
            (0, self.height * cell_size, self.width * cell_size,
             self.height * cell_size + cost_size + padding * 2),
            "black"
        )
        draw.text(
            (padding, self.height * cell_size + padding),
            f"Cost: {self.get_cost(self.hospitals)}",
            fill="white",
            font=font
        )

        img.save(filename)


# Create a new space and add houses randomly
s = Space(height=10, width=20, num_hospitals=3)
for i in range(6):
    s.add_house(random.randrange(s.height), random.randrange(s.width))

# Use local search to determine hospital placement with random start
hospitals = s.hill_climb(image_prefix="hospitals", log=True)


<ipython-input-20-a027f823f912>:35: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  self.hospitals = set(random.sample(self.available_spaces(), self.num_hospitals))


Initial state: cost 37
Found better neighbor: cost 35
Found better neighbor: cost 34
Found better neighbor: cost 33
Found better neighbor: cost 32
Found better neighbor: cost 31
Found better neighbor: cost 30
Found better neighbor: cost 29
Found better neighbor: cost 28
Found better neighbor: cost 27
Found better neighbor: cost 26


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

class SimulatedAnnealingNQueens:
    def _init_(self, n, temp_start, temp_end, alpha, max_iter):
        self.n = n
        self.temp_start = temp_start
        self.temp_end = temp_end
        self.alpha = alpha
        self.max_iter = max_iter

        # Initialize state
        self.current_state = np.random.permutation(n)
        self.current_attacks = self.calculate_attacks(self.current_state)
        self.best_state = self.current_state.copy()
        self.best_attacks = self.current_attacks
        self.temp = temp_start
        self.values = [self.current_attacks]

    def calculate_attacks(self, state):
        """Calculate the number of pairs of queens attacking each other."""
        attacks = 0
        for i in range(self.n):
            for j in range(i + 1, self.n):
                if state[i] == state[j] or abs(state[i] - state[j]) == abs(i - j):
                    attacks += 1
        return attacks

    def generate_neighbor(self, state):
        """Generate a neighbor state by swapping two queens."""
        neighbor = state.copy()
        idx1, idx2 = np.random.choice(self.n, 2, replace=False)
        neighbor[idx1], neighbor[idx2] = neighbor[idx2], neighbor[idx1]
        return neighbor

    def plot_board(self, state, title):
        """Plot the board state."""
        fig, ax = plt.subplots()
        ax.set_xlim(0, self.n)
        ax.set_ylim(0, self.n)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(title)

        # Draw board
        for i in range(self.n):
            for j in range(self.n):
                color = 'white' if (i + j) % 2 == 0 else 'black'
                rect = patches.Rectangle((i, j), 1, 1, linewidth=1, edgecolor='black', facecolor=color)
                ax.add_patch(rect)

        # Draw queens
        for i in range(self.n):
            ax.text(i + 0.5, state[i] + 0.5, 'Q', ha='center', va='center', fontsize=20, color='red')

        plt.gca().invert_yaxis()
        plt.show()

    def anneal(self):
        """Perform the simulated annealing algorithm."""
        for count in range(self.max_iter):
            # Generate a neighboring state
            neighbor = self.generate_neighbor(self.current_state)
            neighbor_attacks = self.calculate_attacks(neighbor)

            # Calculate the difference in the number of attacks
            delta = neighbor_attacks - self.current_attacks

            # Decide whether to accept the neighbor state
            if delta < 0 or np.random.rand() < np.exp(-delta / self.temp):
                self.current_state = neighbor
                self.current_attacks = neighbor_attacks
                if neighbor_attacks < self.best_attacks:
                    self.best_state = neighbor
                    self.best_attacks = neighbor_attacks

            # Update the temperature
            self.temp = max(self.temp * self.alpha, self.temp_end)
            self.values.append(self.best_attacks)

            # Print the current generation and attacks
            print(f"Generation {count}: Attacks {self.current_attacks}")

            # Plot the board every 100 iterations
            self.plot_board(self.current_state, f"Generation {count} - Attacks: {self.current_attacks}")

            # Stop if a solution with zero attacks is found
            if self.best_attacks == 0:
                print(f"Found solution with zero attacks at generation {count}")
                break

        return self.best_state, self.best_attacks, self.values

# Example usage
if __name__ == "_main_":
    # Parameters: size of board, initial temperature, final temperature, cooling rate, maximum iterations
    sa = SimulatedAnnealingNQueens(n=4, temp_start=1000, temp_end=0.1, alpha=0.95, max_iter=1000)
    best_state, best_attacks, values = sa.anneal()
    print("Best state (solution):", best_state)
    print("Number of attacks in the best state:", best_attacks)

## Task 3

In [ ]:
import numpy as np

class HillClimbingOneMax:
    def __init__(self, n):
        self.n = n
        self.current_state = np.random.randint(2, size=n)
        self.best_state = self.current_state.copy()
        self.best_score = self.calculate_score(self.current_state)

    def calculate_score(self, state):
        """Calculate the number of 1s in the bitstring."""
        return np.sum(state)

    def generate_neighbor(self, state):
        """Generate a neighbor state by flipping a random bit."""
        neighbor = state.copy()
        idx = np.random.randint(self.n)
        neighbor[idx] = 1 - neighbor[idx]  # Flip the bit
        return neighbor

    def climb(self, max_iter=1000):
        """Perform the Hill Climbing algorithm."""
        for iteration in range(max_iter):
            # Generate a neighboring state
            neighbor = self.generate_neighbor(self.current_state)
            neighbor_score = self.calculate_score(neighbor)

            # Decide whether to move to the neighbor
            if neighbor_score > self.best_score:
                self.current_state = neighbor
                self.best_score = neighbor_score
                self.best_state = neighbor

            # Print the current iteration and score
            print(f"Iteration {iteration}: Score {self.best_score}")

            # Stop if a solution with maximum score is found
            if self.best_score == self.n:
                print(f"Found solution with maximum score {self.best_score} at iteration {iteration}")
                break

        return self.best_state, self.best_score

# Example usage
if __name__ == "__main__":
    # Parameters: length of bitstring
    hc = HillClimbingOneMax(n=10)
    best_state, best_score = hc.climb(max_iter=1000)
    print("Best state (solution):", best_state)
    print("Number of 1s in the best state:", best_score)


Iteration 0: Score 5
Iteration 1: Score 5
Iteration 2: Score 5
Iteration 3: Score 6
Iteration 4: Score 6
Iteration 5: Score 7
Iteration 6: Score 7
Iteration 7: Score 7
Iteration 8: Score 7
Iteration 9: Score 8
Iteration 10: Score 8
Iteration 11: Score 8
Iteration 12: Score 8
Iteration 13: Score 8
Iteration 14: Score 8
Iteration 15: Score 8
Iteration 16: Score 8
Iteration 17: Score 9
Iteration 18: Score 9
Iteration 19: Score 9
Iteration 20: Score 9
Iteration 21: Score 9
Iteration 22: Score 9
Iteration 23: Score 9
Iteration 24: Score 9
Iteration 25: Score 9
Iteration 26: Score 9
Iteration 27: Score 9
Iteration 28: Score 9
Iteration 29: Score 9
Iteration 30: Score 10
Found solution with maximum score 10 at iteration 30
Best state (solution): [1 1 1 1 1 1 1 1 1 1]
Number of 1s in the best state: 10


## Task 4

In [ ]:
import numpy as np

class HillClimbingScheduler:
    def __init__(self, num_employees, num_shifts, shift_requirements, preferences, max_iterations=1000):
        self.num_employees = num_employees
        self.num_shifts = num_shifts
        self.shift_requirements = shift_requirements
        self.preferences = preferences
        self.max_iterations = max_iterations

    def initialize_schedule(self):
        """Initialize a random schedule."""
        return np.random.randint(0, self.num_employees, size=self.num_shifts)

    def fitness(self, schedule):
        """Evaluate the fitness of a schedule based on requirements and preferences."""
        cost = 0
        shift_counts = np.bincount(schedule, minlength=self.num_employees)

        # Check if all shifts are covered
        for i in range(self.num_shifts):
            if shift_counts[i % self.num_employees] < self.shift_requirements[i]:
                cost += (self.shift_requirements[i] - shift_counts[i % self.num_employees]) * 10  # Penalty for unmet requirements

        # Add preference penalties or bonuses
        for i, employee in enumerate(schedule):
            cost += abs(self.preferences[employee] - i)  # Simplified preference penalty

        return -cost  # We want to maximize fitness, so return negative cost

    def generate_neighbor(self, schedule):
        """Generate a neighbor schedule by changing one shift assignment."""
        neighbor = schedule.copy()
        idx = np.random.randint(0, self.num_shifts)
        neighbor[idx] = np.random.randint(0, self.num_employees)
        return neighbor

    def climb(self):
        """Run the Hill Climbing algorithm to find the best schedule."""
        current_schedule = self.initialize_schedule()
        current_fitness = self.fitness(current_schedule)
        for iteration in range(self.max_iterations):
            neighbor = self.generate_neighbor(current_schedule)
            neighbor_fitness = self.fitness(neighbor)
            if neighbor_fitness > current_fitness:
                current_schedule = neighbor
                current_fitness = neighbor_fitness
            print(f"Iteration {iteration}: Best Fitness {current_fitness}")
        return current_schedule

# Example usage
if __name__ == "__main__":
    num_employees = 10
    num_shifts = 20
    shift_requirements = np.random.randint(1, 3, size=num_shifts)
    preferences = np.random.randint(0, num_employees, size=num_employees)
    hc_scheduler = HillClimbingScheduler(num_employees, num_shifts, shift_requirements, preferences)
    best_schedule = hc_scheduler.climb()
    print("Best schedule:", best_schedule)


Iteration 0: Best Fitness -219
Iteration 1: Best Fitness -219
Iteration 2: Best Fitness -219
Iteration 3: Best Fitness -219
Iteration 4: Best Fitness -211
Iteration 5: Best Fitness -208
Iteration 6: Best Fitness -208
Iteration 7: Best Fitness -191
Iteration 8: Best Fitness -191
Iteration 9: Best Fitness -188
Iteration 10: Best Fitness -161
Iteration 11: Best Fitness -158
Iteration 12: Best Fitness -158
Iteration 13: Best Fitness -158
Iteration 14: Best Fitness -130
Iteration 15: Best Fitness -130
Iteration 16: Best Fitness -130
Iteration 17: Best Fitness -129
Iteration 18: Best Fitness -129
Iteration 19: Best Fitness -129
Iteration 20: Best Fitness -124
Iteration 21: Best Fitness -124
Iteration 22: Best Fitness -124
Iteration 23: Best Fitness -124
Iteration 24: Best Fitness -124
Iteration 25: Best Fitness -124
Iteration 26: Best Fitness -124
Iteration 27: Best Fitness -124
Iteration 28: Best Fitness -124
Iteration 29: Best Fitness -124
Iteration 30: Best Fitness -121
Iteration 31: Best